In [33]:
import json
import numpy as np
from sklearn.metrics import accuracy_score, log_loss

from tools import create_splits
from data import X_train, y_train, X_test
from base_models import Model
from ensemble import Ensemble

In [2]:
display(X_train)
display(X_test)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
0,199.84675,-0.22935,-94.57266,-0.90297,229.65733,-119.73784,4,0.12045,0.50670,13.31960,2.86055,1,951.94743
1,199.76663,0.57961,-98.69548,-0.89087,229.58771,-117.92856,4,-0.28505,0.16832,14.52734,-0.51453,1,949.48540
2,200.40804,-0.03834,-97.64719,-0.89971,230.20797,-121.71234,3,-0.13753,0.71530,9.90651,0.35612,1,953.21660
3,201.38523,0.51942,-105.16820,-0.89226,231.15781,-120.69422,2,0.03872,-0.37550,11.10684,1.32156,1,954.34203
4,199.60130,-0.76663,-94.42670,-0.88284,229.39822,-122.24543,4,0.46721,2.14220,9.72683,-4.96100,1,950.79316
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,199.98926,-0.41084,-96.07238,-0.89343,229.82210,-122.31199,4,-1.46550,-0.52188,9.32058,-5.41452,1,951.91011
4996,198.55424,-1.26159,-100.74327,-0.89437,228.38267,-118.38499,0,-0.50682,1.87498,11.96868,-1.71323,1,942.39957
4997,198.82996,-0.63317,-99.51654,-0.88107,228.61067,-118.17930,0,-0.40644,0.61261,12.00800,-0.66953,1,944.39151
4998,200.17363,-0.36923,-101.12964,-0.89290,229.99216,-120.53125,3,-0.62150,0.97174,10.39919,-6.26654,1,950.30334


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
0,200.16482,-0.26286,-97.71010,-0.89489,229.96954,-120.53342,3,0.78091,-0.09718,9.52442,3.17744,1,951.96903
1,199.54172,0.40905,-100.63736,-0.88896,229.33501,-120.01444,4,-0.46364,-2.63335,12.39610,-1.47704,1,947.38992
2,201.00071,0.14955,-98.64403,-0.88369,230.76183,-120.01524,2,0.89600,-1.21155,11.46621,4.65774,1,955.68154
3,199.79682,2.10760,-99.21798,-0.88805,229.58575,-118.51331,4,-1.34531,-0.37638,9.08931,0.93612,1,949.37509
4,199.12434,0.41191,-103.65685,-0.88177,228.92722,-118.46355,4,2.15090,1.89268,11.01687,-0.31611,1,943.79329
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,198.23317,-0.04672,-97.35953,-0.88230,228.02385,-121.45941,0,1.81188,-1.81356,12.91771,-5.06055,1,942.48610
9996,199.90617,0.42708,-100.51298,-0.89519,229.70554,-122.07958,4,1.46100,-0.36353,9.46819,3.04122,1,949.27436
9997,200.74392,1.06337,-99.16674,-0.88327,230.52623,-122.10346,3,1.81403,-0.62805,12.79995,-3.69950,1,954.13625
9998,200.59772,-0.57665,-100.04636,-0.87773,230.39716,-122.62023,3,-1.62298,-2.15419,15.51824,-1.16185,1,952.96541


In [11]:
with open('params_xgb.json', 'r') as file:
    xgb_params = json.load(file)
with open('params_rf.json', 'r') as file: 
    rf_params = json.load(file)
with open('results_simulation_1.json', 'r') as file:
    simulation_results = json.load(file)

xgb_selected_features = ['x4', 'x8', 'x9', 'x10', 'x11']
gnb_selected_features = ['x2', 'x3', 'x4', 'x6', 'x8', 'x9', 'x10', 'x11']

models = [
    Model(model_type='xgb', selected_features=xgb_selected_features, xgb_params=xgb_params),
    Model(model_type='gnb', selected_features=gnb_selected_features),
    Model(model_type='nn', nn_params=X_train.shape[1]),
    Model(model_type='rf', rf_params=rf_params)
]

In [12]:
simulation_results['1']['best_result']

{'weights': [0.23, 0.23, 0.27, 0.27],
 'mean_accuracy': 0.6868391451068617,
 'std_accuracy': 0.00540453025773596,
 'mean_log_loss': 0.7148769232720507,
 'std_log_loss': 0.006003551213690277}

In [15]:
_X_train, _X_val, _y_train, _y_val = create_splits(X_train, y_train, train_size=0.9)

ensemble = Ensemble(
    models=models,
    model_weights=simulation_results['1']['best_result']['weights']
)
ensemble.fit(_X_train, _y_train, _X_val, _y_val)

In [16]:
y_train_pred, y_train_proba = ensemble.predict(_X_train)
y_val_pred, y_val_proba = ensemble.predict(_X_val)
y_test_pred, y_test_proba = ensemble.predict(X_test)

141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 630us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step


In [21]:
train_accuracy, train_logloss = accuracy_score(_y_train, y_train_pred), log_loss(_y_train, y_train_proba)
val_accuracy, val_logloss = accuracy_score(_y_val, y_val_pred), log_loss(_y_val, y_val_proba)

print("TRAINING SET")
print(f"Accuracy: {train_accuracy:.6f} | Logloss: {train_logloss:.6f} \n")
print("VALIDATION SET")
print(f"Accuracy: {val_accuracy:.6f} | Logloss: {val_logloss:.6f} \n")

TRAINING SET
Accuracy: 0.774000 | Logloss: 0.572882 

VALIDATION SET
Accuracy: 0.708000 | Logloss: 0.683802 



In [30]:
from data import train_raw, label_encoder
y_test = label_encoder.inverse_transform(y_test_pred)
print(f"Y PREDS: {y_test}")

Y PREDS: ['Antrophic' 'OpenAI' 'Antrophic' ... 'OpenAI' 'Mistral' 'OpenAI']


In [34]:
np.savetxt("submission.txt", y_test, fmt="%s")